## NLTK Language Modeling Module

### 1. Preprocessing
Before we train our ngram models it is necessary to make sure the data we put in them is in the right format. Let's say we have a text that is a list of sentences, where each sentence is a list of strings.

In [ ]:
import nltk
from nltk.util import bigrams, trigrams, ngrams

sentences = [['Johann', 'gibt', 'Marie', 'das', 'Manuskript', '.'],
    ['Peter', 'sieht', ',', 'dass', 'Johann', 'Marie', 'das', 'Buch', 'gibt', '.']] 

In [ ]:
sentences[0]

In [ ]:
list(bigrams(sentences[0]))

In [ ]:
list(ngrams(sentences[0],4))

Add special "padding" symbols to the sentence before splitting it into ngrams to indicate how often sentences start or end with a specific token.

In [ ]:
from nltk.lm.preprocessing import pad_both_ends
list(pad_both_ends(sentences[0], n=2)) 

In [ ]:
list(bigrams(pad_both_ends(sentences[0], n=2)))

In [ ]:
padded_bigrams = list(ngrams(pad_both_ends(sentences[0], n=3),n = 3))
padded_bigrams

In [ ]:
from nltk.util import everygrams
list(everygrams(sentences[0], max_len=4))

During training and evaluation our model will rely on a vocabulary that defines which words are **known** to the model. To create this vocabulary we need to **pad** our sentences and then **combine** the sentences into one flat stream of words.

In [ ]:
from nltk.lm.preprocessing import flatten
flt = flatten(pad_both_ends(sent, n=2) for sent in sentences)
flt = list(flt)
flt

In [ ]:
list(set(flt))

Now that we understand what this means for our preprocessing, we can simply import a function that does everything for us. **`padded_everygram_pipeline`**

In [ ]:
from nltk.lm.preprocessing import padded_everygram_pipeline
train, vocab = padded_everygram_pipeline(5, sentences)
list(list(train)[0])

### 2. Training
Having prepared our data we are ready to start training a model. As a simple example, let us train a Maximum Likelihood Estimator (MLE). We only need to specify the highest ngram order to instantiate it.

In [ ]:
help(nltk.lm)

In [ ]:
from nltk.lm import MLE, Laplace
n = 3 # the highest ngram order
lm = Laplace(n)  #  Maximum Likelihood Estimator 

In [ ]:
lm.order

This automatically creates an empty vocabulary

In [ ]:
len(lm.vocab)

In [ ]:
train, vocab = padded_everygram_pipeline(3, sentences)
lm.fit(train, vocab)
print(list(lm.vocab))

In [ ]:
list(lm.vocab)

The vocabulary helps us handle words that have not occurred during training.

In [ ]:
lm.vocab.lookup(sentences[0])

Items that are not seen during training are mapped to the vocabulary's **unknown label** token. This is **`<UNK>`** by default.

In [ ]:
lm.vocab.lookup(["Thomas", 'spielen', 'Marie', 'das', 'Manuskript', '.'])

In [ ]:
 print(lm.counts)

In [ ]:
lm.counts['Johann']

In [ ]:
lm.counts[['Johann']]['gibt']

In [ ]:
lm.score('gibt', ['Johann'])

In [ ]:
lm.score('das', ['Marie'])

**Practical Issues:** To avoid underflow when working with many small score values it makes sense to take their logarithm. also adding is faster than multiplying.

$log(p_1 x p_2 x p_3 x p_4) = log(p_1) + log(p_2) + log(p_3) + log(p_4)$

In [ ]:
lm.logscore('gibt', ['Johann'])

Building on this method, we can also evaluate our model's **cross-entropy** and **perplexity** with respect to sequences of ngrams.

$H(W)=\frac{1}{n}logp(W_1^n)$


$perplexity(W) = 2^{H(W)}$

In [ ]:
lm.perplexity(["Manuskript"])

In [ ]:
test = [('Johann', 'sieht'), ('das', 'Buch')]
lm.entropy(test)

In [ ]:
lm.score('sieht', ['Johann'])

In [ ]:
lm.score('Buch', ['das'])

The best language model is one that best predicts an unseen test set

### Smoothing
#### Add-one smoothing (Laplace)
$P_{Laplace}(w_i | w_{i-1}) = \frac{c(w_{i-1},w_i) + 1}{c(w_{i-1}) + V}$

$v$ = vocabulary size

In [ ]:
from nltk.lm.models import Laplace

#### Kneser–Ney smoothing


In [ ]:
from nltk.lm.models import KneserNeyInterpolated

#### Unknown Contexts
*Note*: to enable unknown contexts, run `lm.counts.update([[('<UNK>', '<UNK>')]])` after training.

In [ ]:
lm.counts.update([[('<UNK>', '<UNK>')]])

### Perplexity
We use the so-called perplexity as a measure for comparing language models. It measures how **surprised** the model is by a new text (i.e. smaller values are better).
Lower perplexity = better model

In [ ]:
test = [('Johann', 'sieht'), ('das', 'Buch')]

lm.perplexity(test)

In [ ]:
test = [('Johann', 'gibt'), ('das', 'Buch')]

lm.perplexity(test)

## Language Generation

<div class="alert alert-info">

**generate(num_words=1, text_seed=None, random_seed=None)**
    
Generate words from the model.

Parameters:
- **num_words (int)** – How many words to generate. By default 1.
- **text_seed** – Generation can be conditioned on preceding context.
- **random_seed** – A random seed or an instance of random.Random. If provided (Provide random_seed if you want to consistently reproduce the same text all other things being equal.) 

Note that an ngram model is restricted in how much preceding context it can take into account. For example, a trigram model can only condition its output on 2 preceding words. If you pass in a 4-word context, the first two words will be ignored.
</div>

In [ ]:
lm.generate(9, random_seed=5)

In [ ]:
from nltk.corpus import  brown

In [ ]:
len(brown.sents())

In [ ]:
# Brown corpus
train_brown, vocab_brown = padded_everygram_pipeline(n, brown.sents())
lm_brown = MLE(n)
lm_brown.fit(train_brown, vocab_brown)

In [ ]:
lm_brown.generate(10,text_seed=['Germany'])

### Corpora

The Universal Treebanks corpus has subcorpora for languages:
You can import a corpus by its filename `std/de/de-universal-train.conll`:

In [ ]:
from nltk.corpus import universal_treebanks as ut
test_sentences = ut.sents("std/de/de-universal-train.conll")[:100]
test_sentences[:10]

You can get the brown subcorpus for a language by:

In [ ]:
for lang in ['de', 'fr', 'sv', 'id']:
    filename = f'std/{lang}/{lang}-universal-train.conll'
    print(filename)

In [ ]:
ut.sents("std/sv/sv-universal-train.conll")[:100]